In [3]:
!ls

assignment_2.ipynb  brown.txt  junk.ipynb  question.pdf


In [4]:
import string, re
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm.auto import tqdm
import numpy as np
from collections import Counter
import random

In [5]:
class txt_data():
    '''
    Master class to load dataset
    '''
    def __init__(self,path):
        self.path = path
        
        # list of sentences of list of tuples defining pos_tag and words.
        self.data = self.___sent_to_tags___(self.___open___(self)) 
        random.shuffle(self.data)
        self.train =  self.data[:int(len(self.data)*0.8)]
        self.val = self.data[int(len(self.data)*0.8):]
        
        self.pos_tags = [i for i in set([j[0] for i in self.data for j in i])] #list of unique pos_tags
        self.vocab = [i for i in set([j[1] for i in self.data for j in i])] # list of vocab
            
    
    def ___open___(self,path):
        '''
        To read a text file
        path: path to the text file
        '''
        with open(self.path, "r") as f:
            return f.readlines()
    
    def ___sent_to_tags___(self,sents):
        '''
        '''
        dummy = []
        for i in tqdm(sents):
            d = []
            for j in i.split():
                try: d.append((j.split('_')[1],j.split('_')[0].lower()))
                except: pass #print(f"--{i}--{j}\n")
            dummy.append(d)
        return dummy

In [6]:
path = "brown.txt"
brown = txt_data(path)

In [7]:
len(brown.data), len(brown.train), len(brown.val), len(brown.train)+len(brown.val)

(55145, 44116, 11029, 55145)

In [127]:
def prob(dict_): return {k:np.array(v)/sum(v) for k,v in dict_.items()}

In [125]:
def e_matrix(data,vocab):
    dummy = {}
    data = [i for j in data for i in j] # list of word and tag pair
    print(data[:4])
    for i in data:
        if i[0] in dummy.keys():
            dummy[i[0]].append(i[1])
        else: dummy[i[0]] = [i[1]]
    e_matrix = {}
    for t,v in tqdm(dummy.items()): 
        d = []
        count = Counter(v)
        for j in vocab:
            d.append(count[j])
        e_matrix[t] = d
    return e_matrix
e_prob = prob(e_matrix(brown.data, brown.vocab))

[('VB', 'share'), ('NN', 'bath'), (',', ','), ('RB', 'maybe')]


In [129]:
def t_matrix(data, pos_tags, n_gram=2):
    dummy = {}
    data = [[j[0]  for j in i] for i in data] # list of tag 
    print(data[:2],'\n')
    start = Counter([j for i in data for j in i])
    tags = [i[j:j+n_gram] for i in data for j in range(len(i) - n_gram + 1)] #list of n_grams of tags
#     print(tags[:10])
    tags_ = {i:[] for i in pos_tags}
    for i in tags:
        tags_[i[0]].append(i[1])
    t_matrix = {}
    t_matrix["start"] = [start[i] for i in pos_tags]
    for tp,tf in tqdm(tags_.items()): 
        d = []
        count = Counter(tf)
        for j in pos_tags:
            d.append(count[j])
        t_matrix[tp] = d
    return t_matrix
t_prob = prob(t_matrix(brown.data, brown.pos_tags, n_gram=2))

[['VB', 'NN', ',', 'RB', '--', 'CC', 'VB', 'NN', '.'], ['AT', 'NN', 'IN', 'NNS', 'IN', 'AT', 'NN-TL', 'IN-TL', 'NP-TL', 'HV', 'VBN', 'AT', 'NN', 'WDT', 'RB', 'VBZ', 'DTI', 'IN', 'DTS', 'NNS', '.']] 

